In [67]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

In [68]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

84125825/84125825 [==============================] - 14s 0us/step


['imdb.vocab', 'imdbEr.txt', 'test', 'README', 'train']

In [69]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['labeledBow.feat',
 'pos',
 'unsup',
 'neg',
 'urls_pos.txt',
 'unsupBow.feat',
 'urls_unsup.txt',
 'urls_neg.txt']

In [70]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [71]:
batch_size = 1024
seed = 123
train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [73]:
# for text_batch, label_batch in train_ds.take(1):
#     for i in range(5):
#         print(label_batch[i].numpy(), text_batch.numpy()[i])

In [74]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [79]:
embedding_layer = tf.keras.layers.Embedding(1000, 12)

In [80]:
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[ 0.04083678, -0.04414196, -0.0334352 ,  0.01706639,  0.01619056,
         0.04879651, -0.04826605,  0.0293254 ,  0.02902663,  0.00482672,
         0.0027861 ,  0.01292069],
       [ 0.00456624,  0.01342162, -0.03140541,  0.04949732, -0.02804502,
        -0.01975658,  0.01508491, -0.00975075, -0.02492904, -0.00084225,
        -0.03479319, -0.04870179],
       [ 0.01455637, -0.0343691 , -0.04432988,  0.02619549, -0.03287293,
         0.01760976,  0.0477837 ,  0.01766977, -0.0267094 , -0.01795704,
        -0.04491169,  0.03071668]], dtype=float32)

In [88]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 50000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [89]:
embedding_dim=384

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(embedding_dim, activation='relu'),
  Dense(1)
])

In [90]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [91]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [92]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
20/20 [==============================] - 8s 289ms/step - loss: 0.6797 - accuracy: 0.5036 - val_loss: 0.6410 - val_accuracy: 0.5178
Epoch 2/15
20/20 [==============================] - 1s 61ms/step - loss: 0.5607 - accuracy: 0.6776 - val_loss: 0.4766 - val_accuracy: 0.7572
Epoch 3/15
20/20 [==============================] - 1s 60ms/step - loss: 0.3712 - accuracy: 0.8338 - val_loss: 0.3860 - val_accuracy: 0.8186
Epoch 4/15
20/20 [==============================] - 1s 61ms/step - loss: 0.2456 - accuracy: 0.8992 - val_loss: 0.3673 - val_accuracy: 0.8348
Epoch 5/15
20/20 [==============================] - 1s 65ms/step - loss: 0.1640 - accuracy: 0.9396 - val_loss: 0.3858 - val_accuracy: 0.8330
Epoch 6/15
20/20 [==============================] - 1s 60ms/step - loss: 0.1082 - accuracy: 0.9643 - val_loss: 0.4215 - val_accuracy: 0.8316
Epoch 7/15
20/20 [==============================] - 1s 61ms/step - loss: 0.0716 - accuracy: 0.9791 - val_loss: 0.4859 - val_accuracy: 0.8324
Epoch 8/15
2

In [93]:
#docs_infra: no_execute
%load_ext tensorboard
%tensorboard --logdir logs

In [94]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [96]:
# vocab

In [97]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [98]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass